## 1. 矩阵1和矩阵2每行对应相同样本、信号彼此独立
### 每个信号的表达式：y = ax + b

In [68]:
import numpy as np

# 定义矩阵1和矩阵2
matrix1 = np.array([
    [1, 2, 3, 4, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 6, 7]
])
matrix2 = np.array([
    [0.9, 2.3, 2.9, 4.2, 7.1],
    [1.8, 3.1, 3.9, 5.2, 6.2],
    [2.6, 4.1, 5.2, 6.1, 5.3]
])

# 获取矩阵形状 n_samples=3， n_signals=5
n_samples, n_signals = matrix1.shape

# 初始化校准参数和校准后的矩阵
a_params = []
b_params = []
calibrated_matrix2 = np.zeros_like(matrix2)

# 逐信号计算校准参数并校准数据
for signal_idx in range(n_signals):
    # 提取对应信号的数据
    x = matrix2[:, signal_idx]
    y = matrix1[:, signal_idx]
    # 一次方程线性拟合，计算校准参数a，b
    a, b = np.polyfit(x, y, 1)
    a_params.append(a)
    b_params.append(b)
    
    # 应用校准公式校准批次2数据
    calibrated_matrix2[:, signal_idx] = a * x + b

# 输出校准参数和校准后的数据
print("校准参数a:", a_params)
print("校准参数b:", b_params)
print("校准后的批次2数据:")
print(calibrated_matrix2)

校准参数a: [1.1751152073732716, 1.10655737704918, 0.8646616541353379, 1.0516605166051662, -1.1111111111111116]
校准参数b: [-0.07603686635944676, -0.5040983606557359, 0.541353383458649, -0.433579335793358, 12.888888888888886]
校准后的批次2数据:
[[0.98156682 2.04098361 3.04887218 3.98339483 5.        ]
 [2.03917051 2.92622951 3.91353383 5.03505535 6.        ]
 [2.97926267 4.03278689 5.03759398 5.98154982 7.        ]]


## 2. 矩阵1和矩阵2每行对应不同样本（两次检测用了完全不同的样本）
### 利用均值和标准差校准
### 校准后的批次2信号 = (批次2信号 − μ2) / σ2 * σ1 + μ1
### μ,σ: 每个信号的均值和标准差

In [51]:
import numpy as np

# 定义矩阵1和矩阵2
matrix1 = np.array([
    [1, 2, 3, 4, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 6, 7]
])
matrix2 = np.array([
    [0.9, 2.3, 2.9, 4.2, 7.1],
    [1.8, 3.1, 3.9, 5.2, 6.2],
    [2.6, 4.1, 5.2, 6.1, 5.3]
])

# 计算每列的均值和标准差
mean1 = np.mean(matrix1, axis=0)
std1 = np.std(matrix1, axis=0)
mean2 = np.mean(matrix2, axis=0)
std2 = np.std(matrix2, axis=0)

# 校准批次2数据
calibrated_matrix2 = (matrix2 - mean2) / std2 * std1 + mean1

# 输出校准结果
print("批次1信号的均值:", mean1)
print("批次1信号的标准差:", std1)
print("批次2信号的均值:", mean2)
print("批次2信号的标准差:", std2)
print("校准后的批次2数据:")
print(calibrated_matrix2)

批次1信号的均值: [2. 3. 4. 5. 6.]
批次1信号的标准差: [0.81649658 0.81649658 0.81649658 0.81649658 0.81649658]
批次2信号的均值: [1.76666667 3.16666667 4.         5.16666667 6.2       ]
批次2信号的标准差: [0.69442222 0.7363574  0.94162979 0.77602978 0.73484692]
校准后的批次2数据:
[[0.98097966 2.03901235 3.04617903 3.9829256  7.        ]
 [2.03919309 2.92607787 3.913289   5.03507153 6.        ]
 [2.97982725 4.03490978 5.04053196 5.98200287 5.        ]]


## 3. 矩阵1和矩阵2每行对应不同样本且信号之间相互关联
### 假设每个信号之间的关系是线性的，因此找出一个校准矩阵，使得校准后的批次2数据尽量接近批次1数据
### 用最小二乘法来找出校准矩阵
### 将把matrix2每一列与matrix1的每一列做线性拟合

In [55]:
import numpy as np

# 定义批次1和批次2的数据
matrix1 = np.array([
    [1, 2, 3, 4, 5], 
    [2, 3, 4, 5, 6], 
    [3, 4, 5, 6, 7]
])

matrix2 = np.array([
    [0.9, 2.3, 2.9, 4.2, 7.1],
    [1.8, 3.1, 3.9, 5.2, 6.2],
    [2.6, 4.1, 5.2, 6.1, 5.3]
])


# 计算校准矩阵
A = matrix2.T  # 转置，使得每一列代表一个信号
B = matrix1.T  # 转置，使得每一列代表一个信号

# 使用最小二乘法求解校准系数
calibration_matrix, residuals, rank, s = np.linalg.lstsq(A, B, rcond=None)

# 输出校准参数
print("批次2到批次1的校准参数：")
print(calibration_matrix)

# 校准批次2数据
calibrated_matrix2 = np.dot(A, calibration_matrix)

# 输出校准后的批次2数据
print("校准后的批次2数据：")
print(calibrated_matrix2.T)  # 转置回原始形状


批次2到批次1的校准参数：
[[ 0.29109633  0.03223406 -0.2266282 ]
 [ 0.4298127   0.82276796  1.21572321]
 [ 0.06146037  0.11705364  0.1726469 ]]
校准后的批次2数据：
[[1.19544653 2.25392847 2.84004284 3.83253891 5.05736267]
 [1.81433244 3.10463893 3.91095274 5.12780364 5.95040747]
 [2.43321835 3.9553494  4.98186264 6.42306837 6.84345226]]


## 4. 统一命名规则后将文件存储到"/database/interview/"
### 构建命名规则：
第一类：面试情况表
### 命名规则设定为：<人名>_<创建时间>.txt

## 4. 统一命名规则后将文件存储到"/database/interview/"

In [72]:
import shutil
from pathlib import Path

def process_file(file_path, root_dir, output_dir):
    # 将路径转换为 Path 对象，方便处理
    file_path = Path(file_path)
    root_dir = Path(root_dir)
    output_dir = Path(output_dir)
    
    # 获取文件名和扩展名
    file_name, file_extension = file_path.stem, file_path.suffix
    
    # 将文件后缀改为 .txt
    new_file_name = file_name + ".txt"
    
    # 替换原始路径中的 root_dir 为 output_dir，并保持后续路径结构不变
    relative_path = file_path.relative_to(root_dir)
    
    # 构建新的文件路径
    new_file_path = (output_dir / relative_path).as_posix() # 默认保存为Linux格式
    print(new_file_path)
    
    # 创建目标路径的父目录（如果不存在）
    # new_file_path.parent.mkdir(parents=True, exist_ok=True)
    
    # 复制文件到新的路径
    # shutil.copy(file_path, new_file_path.with_suffix(".txt"))
    
    # 输出新文件路径
    # print(f"文件已保存到：{new_file_path.with_suffix('.txt')}")

# 示例文件路径
file1 = "/root/interview_人名1.txt"
file2 = "/root/主管1/人名1-第二次-202303.txt"
file3 = "/root/面试评价/人名2-2022.1.doc"

# 原始根目录
root_dir = "/root"

# 输出目录
output_dir = "/database/interview/"

# 处理文件
process_file(file1, root_dir, output_dir)
process_file(file2, root_dir, output_dir)
process_file(file3, root_dir, output_dir)


/database/interview/interview_人名1.txt
/database/interview/主管1/人名1-第二次-202303.txt
/database/interview/面试评价/人名2-2022.1.doc
